In [3]:
import ecdsa
import hashlib

def create_key_pair():
    private_key = ecdsa.SigningKey.generate(curve=ecdsa.SECP256k1)
    public_key = private_key.get_verifying_key()
    return private_key, public_key

names = ["System", "Alice", "Bob", "Carol", "David", "Eve", "Frank"]

users = {}
for name in names:
    private_key, public_key = create_key_pair()
    users[name] = {
        "private_key": private_key,
        "public_key": public_key,
        "old_private_key": None,
        "old_public_key": None
    }


balances = {name: 0 for name in names}
balances["Alice"] = 100


def initiate_transaction(sender, receiver, amount):
    sender_private_key = users[sender]["private_key"]
    sender_public_key = users[sender]["public_key"]
    receiver_private_key = users[receiver]["private_key"]
    receiver_public_key = users[receiver]["public_key"]


    if balances[sender] >= amount:
        balances[sender] -= amount
        balances[receiver] += amount
        sender_balance = balances[sender]
        receiver_balance = balances[receiver]


        receiver_old_public_key = users[receiver]["old_public_key"]


        trans_str = f"Sender: {sender}\nReceiver: {receiver}\nAmount: {amount}\n"
        transaction_hash = hashlib.sha256(trans_str.encode()).hexdigest()


        signature = sender_private_key.sign(transaction_hash.encode())


        trans_verification = sender_public_key.verify(signature, transaction_hash.encode())

        transaction_data = {
            "sender": sender,
            "receiver": receiver,
            "amount": amount,
            "sender_balance": sender_balance,
            "receiver_balance": receiver_balance,
            "transaction_hash": transaction_hash,
            "signature": signature,
            "sender_public_key": sender_public_key,
            "receiver_new_public_key": receiver_public_key,
            "trans_verification": trans_verification
        }

        if receiver_old_public_key:
            transaction_data["receiver_old_public_key"] = receiver_old_public_key

        return transaction_data


def merkle_root(transactions):
    if len(transactions) == 0:
        return None
    if len(transactions) == 1:
        return transactions[0]

    new_transactions = []
    for i in range(0, len(transactions), 2):
        if i + 1 < len(transactions):
            combined = transactions[i] + transactions[i + 1]
            hash_combined = hashlib.sha256(combined.encode()).hexdigest()
            new_transactions.append(hash_combined)
        else:

            new_transactions.append(transactions[i])

    return merkle_root(new_transactions)


genesis_trans = [
    ("System", "Alice", 100)
]

genesis_trans_hash = []
for sender, receiver, amount in genesis_trans:
    transaction_data = initiate_transaction(sender, receiver, amount)
    if transaction_data:
        genesis_trans_hash.append(transaction_data["transaction_hash"])

genesis_merkle_root = merkle_root(genesis_trans_hash)


print("Genesis Block:")
print("Block Header: 0")
print("Block Number: 0")
print("Previous Block Header: None")
print("Merkle Root:", genesis_merkle_root)
print("Transactions:")
for transaction in genesis_trans:
    print("  Sender:", transaction[0])
    print("  Receiver:", transaction[1])
    print("  Amount:", transaction[2])
    print()


def generate_block(block_num, prev_block_hash, transactions):
    transaction_hashes = [transaction["transaction_hash"] for transaction in transactions if transaction]
    merkle_root1 = merkle_root(transaction_hashes)
    block_head = f"Block Number: {block_num}\nPrevious Block Header: {prev_block_hash}\nMerkle Root: {merkle_root1}"
    block_header = hashlib.sha256(block_head.encode()).hexdigest()

    print(f"Block Header:", block_header)
    print("Block Number:", block_num)
    print("Previous Block Header:", prev_block_hash)
    print("Merkle Root:", merkle_root1)
    print()
    print("Transactions:")
    print()
    for transaction in transactions:
        if transaction:
            print("  Transaction from", transaction["sender"], "to", transaction["receiver"])
            print("  Amount:", transaction["amount"])
            print("  Sender Balance:", transaction["sender_balance"])
            print("  Receiver Balance:", transaction["receiver_balance"])
            print("  Sender's Public Key:", transaction["sender_public_key"].to_string().hex())
            if "receiver_old_public_key" in transaction:
                print("  Receiver's Old Public Key:", transaction["receiver_old_public_key"].to_string().hex())
            print("  Receiver's New Public Key:", transaction["receiver_new_public_key"].to_string().hex())
            print("  Transaction Hash:", transaction["transaction_hash"])
            print("  Transaction Signature:", transaction["signature"].hex())
            print("  Verification:", transaction["trans_verification"])


            print()
            print()

prev_block_hash = None
for block_num in range(1, 4):
    block_transactions = [
        initiate_transaction("Alice", "Bob", 20),
        initiate_transaction("Bob", "Carol", 5),
        initiate_transaction("Carol", "David", 2),
        initiate_transaction("David", "Eve", 1)
    ]
    generate_block(block_num, prev_block_hash, block_transactions)
    prev_block_hash = hashlib.sha256(str(block_num).encode()).hexdigest()

print("Remaining Balances:")
for user_name in balances:
    if user_name != "System":  # Exclude the System user
        print(f"{user_name}: {balances[user_name]} coins")


Genesis Block:
Block Header: 0
Block Number: 0
Previous Block Header: None
Merkle Root: None
Transactions:
  Sender: System
  Receiver: Alice
  Amount: 100

Block Header: 78f85362bc566bc6c48ea3bdad1328f0220ff83a956bede1777bd12a02fa7d0c
Block Number: 1
Previous Block Header: None
Merkle Root: 9e59d7d2915b498bffaa149e4f962f6bc0346ef1587658fc06bbb20dcb28a10e

Transactions:

  Transaction from Alice to Bob
  Amount: 20
  Sender Balance: 80
  Receiver Balance: 20
  Sender's Public Key: 923e3b95ea852406e4b3e0e5d17c60dedc378e8d2572ec6c290d8c720f1cb99ea03a006c0dbcc91c1f88b4e4db01f998d0bc4d85c5eba7576053f39395761a53
  Receiver's New Public Key: 38bbb0b3d73920fa6ffca02afe46c1b127a6626e7bba21c4dfc35b17626f4a74ed17249b3ba36b6621d6da2fbc528437854cba5f540a4771950709eb08303d75
  Transaction Hash: 51445009a3f9d8d073a41a5d54da7b38de93a3e3dc47162cdc937a8757337aab
  Transaction Signature: 0d9f71f6bb8751887069ba61894a16e5e486e3b8ebc63dcea7df6d05ebe241a29ecf96c513b92d0a973e5d5e2055b54fb313b0839312dc456d0f9